In [1]:
import pickle, os
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
import keras

Using TensorFlow backend.
/Users/charles/anaconda/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/charles/anaconda/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/charles/anaconda/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/charles/anaconda/envs/

# Data Importation and test-valid-train split

In [5]:
#
# Data importation
#
print('Loading data...')
X = np.load('./data/excellent_unoriented/obs.npy')
y = pickle.load( open( "./data/excellent_unoriented/labels.pkl", "rb" ) )

X_synth = np.load('./data/mixed_synthetic/signal_synthetic.npy')
y = 

#
# Train-Test split
#
X_i, X_test, y_i, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)
X_train, X_valid, y_train, y_valid = train_test_split(X_i, y_i, test_size=0.20, random_state=42, shuffle=True)

print(X_train.shape)

#
# Reshaping
# we need to expand to have 3D tensors as input of the 1D CNN

X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)

print('Shape is {}'.format(X_train.shape))
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

nb_class = 27
y_train = keras.utils.to_categorical(y_train, nb_class)
y_test = keras.utils.to_categorical(y_test, nb_class)
y_valid = keras.utils.to_categorical(y_valid, nb_class)

Loading data...
(205, 1000)
Shape is (205, 1000, 1)
205 train sequences
65 test sequences


# Building model

In [36]:
#%%
print('Build model...')
model = Sequential()

# create the model
model.add(Conv1D(4,2,activation='relu',kernel_initializer='glorot_uniform',input_shape=(X_train.shape[1],1)))
model.add(MaxPooling1D())
model.add(Dropout(0.2))

#model.add(Conv1D(2,2,activation='relu',kernel_initializer='glorot_uniform'))
#model.add(MaxPooling1D())
#model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(100, activation='relu'))

model.add(Dense(nb_class, activation='softmax'))


Build model...


# Initialize model

In [37]:
early_stopping = EarlyStopping(monitor='val_loss', patience=0)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 999, 4)            12        
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 499, 4)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 499, 4)            0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1996)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 100)               199700    
_________________________________________________________________
dense_19 (Dense)             (None, 27)                2727      
Total params: 202,439
Trainable params: 202,439
Non-trainable params: 0
_________________________________________________________________
None

# Fit

In [38]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=8,callbacks=[early_stopping])

Train on 205 samples, validate on 52 samples
Epoch 1/100
205/205 [==============================] - 2s 9ms/step - loss: 2.8120 - acc: 0.2049 - val_loss: 2.1303 - val_acc: 0.5192
Epoch 2/100
205/205 [==============================] - 0s 2ms/step - loss: 1.7272 - acc: 0.5610 - val_loss: 0.9318 - val_acc: 0.7885
Epoch 3/100
205/205 [==============================] - 0s 2ms/step - loss: 0.9714 - acc: 0.7317 - val_loss: 0.4990 - val_acc: 0.8654
Epoch 4/100
205/205 [==============================] - 0s 2ms/step - loss: 0.5649 - acc: 0.8634 - val_loss: 0.3876 - val_acc: 0.9231
Epoch 5/100
205/205 [==============================] - 0s 2ms/step - loss: 0.3916 - acc: 0.9073 - val_loss: 0.3293 - val_acc: 0.8846
Epoch 6/100
205/205 [==============================] - 0s 2ms/step - loss: 0.3327 - acc: 0.8829 - val_loss: 0.3070 - val_acc: 0.8846
Epoch 7/100
205/205 [==============================] - 0s 2ms/step - loss: 0.2883 - acc: 0.9024 - val_loss: 0.3407 - val_acc: 0.8269


# Evaluation on test subset

In [39]:
scores_train = model.evaluate(X_train, y_train, verbose=0)
print("Train accuracy: %.2f%%" % (scores_train[1]*100))
scores_valid = model.evaluate(X_valid, y_valid, verbose=0)
print("Valid accuracy: %.2f%%" % (scores_valid[1]*100))
scores_test = model.evaluate(X_test, y_test, verbose=0)
print("TestAccuracy: %.2f%%" % (scores_test[1]*100))

Train accuracy: 91.22%
Valid accuracy: 82.69%
TestAccuracy: 90.77%
